カテゴリーに分類するモデル構築　

In [4]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [5]:
text_list = ["ドミノを倒します。", "プリンがあります。", "大阪へ行きます。", "スライムがいます。", "痛い、頭が。", "崩す、体勢を。"]
label_listA = [6, 6, 0, 6, 6, 6]
label_listB = [0, 0, 0, 0, 1, 1] # map 0 box 1

In [6]:
category_list = [
    'movement', #0
    'combat',   #1
    'take',     #2
    'use',      #3
    'find',     #4
    'buy',      #5
    'unknown'   #6
]
# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

max_length = 20
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./my_data/{category}*'):
        lines = open(file).read().splitlines()
        for text in lines:
            print(text)
            print('----')
            encoding = tokenizer(
                text,
                max_length=max_length,
                padding='max_length',
                truncation=True
            )
            encoding['labels'] = label
            encoding = { k: torch.tensor(v) for k, v in encoding.items()}
            dataset_for_loader.append(encoding)


print(dataset_for_loader[0])

# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
print(n)

100%|██████████| 7/7 [00:00<00:00, 502.48it/s]

海に行く。
----
##へ行く。
----
ハワイへ行く。
----
町まで移動する。
----
道を進む。
----
参る。
----
伺う。
----
まいる。
----
うかがう。
----
出発する。
----
足を運ぶ。
----
歩を運ぶ。
----
ダンジョンに向かう。
----
赴く。
----
道を前進する。
----
歩く。
----
走る。
----
旅立つ。
----
訪問する。
----
現場に出向く。
----
町にたどる。
----
到達する。
----
に行ってくる。
----
##へ##てくる。
----
##に##でくる。
----
##へ##でくる。
----
##に行って来る。
----
##へ##て来る。
----
##に##で来る。
----
##へ##で来る。
----
尋ねる。
----
飛んでいく。
----
ゴブリンと戦闘する。
----
ゴブリンと戦う。
----
ゾンビを倒す。
----
敵を殲滅する。
----
たたかう。
----
破る。
----
打ち破る。
----
掃滅する。
----
殺す。
----
敵機を撃破する。
----
打倒す。
----
打倒する。
----
勝利を収める。
----
討ち破る。
----
打ち負かす。
----
たたき伏せる。
----
薙ぎ倒す。
----
殴り倒す。
----
倒してくる。
----
アイテムをとる。
----
薬を採取する。
----
標本を採る。
----
素材を採取する。
----
収集する。
----
回収する。
----
抽出する。
----
収穫する。
----
採集する。
----
サンプルをとる。
----
集める。
----
採ってくる。
----
捕まる。
----
使う。
----
使用する。
----
飲む。
----
利用する。
----
装備する。
----
用いる。
----
扱う。
----
馬に乗る。
----
笛を吹く。
----
探す。
----
見つける。
----
発見する。
----
探してくる。
----
見つけて来る。
----
発見した。
----
見つけた。
----
捜す。
----
探知する。
----
探査する。
----
探索する。
----
サーチする。
----
捜索する。
----


In [7]:
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする
dataloader_train = DataLoader(dataset_train, batch_size=2, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=16)
dataloader_test = DataLoader(dataset_test, batch_size=16)

import pytorch_lightning as pl
class BertForSequenceClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()

        # 引数のnum_labelsとlrを保存
        # 例えば、self.hparams.lrでlrにアクセスできる
        # チェックポイント作成時にも自動で保存される
        self.save_hyperparameters()

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

    # 学習データのミニバッチ（'batch')が与えられた時に損失を出力する関数を書く
    # batch_idxはミニバッチの番号であるが今回は使わない
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる
        return loss
    
    # 検証データのミニバッチが与えられたときに、
    # 検証データを評価する指標を計算する関数を書く
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる

    # テストデータのミニバッチが与えられたときに、
    # テストデータを評価する指標を計算する関数を書く
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = (labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) # 精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる

    # 学習に用いるオプティマイザを返す関数を書く
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
    
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    #gpus=1,
    max_epochs=10,
    callbacks = [checkpoint]
)
# num_labels = 9 
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=7, lr=1e-5
)

# ファインチューニングを行う
trainer.fit(model, dataloader_train, dataloader_val)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a CUDA device ('NVIDIA GeForce RTX 3080 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                          | Params | Mode
-----------------------------------------------------------------
0 | bert_sc | BertForSequ

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/yozora98/Works/python/nlp2/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


/home/yozora98/Works/python/nlp2/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
/home/yozora98/Works/python/nlp2/.venv/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (37) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 9: 100%|██████████| 37/37 [00:00<00:00, 51.54it/s, v_num=4]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 37/37 [00:00<00:00, 51.47it/s, v_num=4]


In [8]:
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  /home/yozora98/Works/python/nlp2/model/epoch=8-step=333.ckpt
ベストモデルの検証データに対する損失:  tensor(0.6026, device='cuda:0')


In [9]:

%load_ext tensorboard
%tensorboard --logdir="~/Project/nlp2/lightning_logs/"

In [254]:
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

Restoring states from the checkpoint path at /home/yozora98/Project/nlp2/model/epoch=9-step=370.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/yozora98/Project/nlp2/model/epoch=9-step=370.ckpt


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 106.88it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy             0.807692289352417
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy: 0.81


In [255]:
# Pytorch Lightning モデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(best_model_path)

# Transformers 対応のモデルを ./model_transformersに保存
model.bert_sc.save_pretrained('./model_transformers')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [256]:
bert_sc = BertForSequenceClassification.from_pretrained('./model_transformers')

In [257]:
# 推論 category

encoding = tokenizer(
    text_list,
    padding = 'longest',
    return_tensors='pt'
)
#encoding = { k: v.cuda() for k, v in encoding.items()}
#labels = torch.tensor(label_list).cuda()
encoding = { k: v for k, v in encoding.items()}
labels = torch.tensor(label_listA)
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits
labels_predictedB = scores.argmax(-1)
num_correct = (labels_predictedB==labels).sum().item()
accuracy = num_correct/labels.size(0)

print('size:')
print(scores.size())
print('predicted labels:')
print(labels_predictedB)
print('accuracy:')
print(accuracy)

size:
torch.Size([6, 9])
predicted labels:
tensor([6, 6, 0, 6, 6, 6])
accuracy:
1.0


次はモデル２の構築 - map上のオブジェクトか？box内のオブジェクトか？

In [258]:
category_list = [
    'map',
    'box'
]

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

max_length = 20
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./my_data2/{category}*'):
        lines = open(file).read().splitlines()
        for text in lines:
            encoding = tokenizer(
                text,
                max_length=max_length,
                padding='max_length',
                truncation=True
            )
            encoding['labels'] = label
            encoding = { k: torch.tensor(v) for k, v in encoding.items()}
            dataset_for_loader.append(encoding)


print(dataset_for_loader[0])

# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
print(n)

100%|██████████| 2/2 [00:00<00:00, 216.83it/s]

{'input_ids': tensor([   2,  295,    7, 3488,    8,    3,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'labels': tensor(0)}
80


In [259]:
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする
dataloader_train = DataLoader(dataset_train, batch_size=2, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=16)
dataloader_test = DataLoader(dataset_test, batch_size=16)

import pytorch_lightning as pl
class BertForSequenceClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()

        # 引数のnum_labelsとlrを保存
        # 例えば、self.hparams.lrでlrにアクセスできる
        # チェックポイント作成時にも自動で保存される
        self.save_hyperparameters()

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

    # 学習データのミニバッチ（'batch')が与えられた時に損失を出力する関数を書く
    # batch_idxはミニバッチの番号であるが今回は使わない
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる
        return loss
    
    # 検証データのミニバッチが与えられたときに、
    # 検証データを評価する指標を計算する関数を書く
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる

    # テストデータのミニバッチが与えられたときに、
    # テストデータを評価する指標を計算する関数を書く
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = (labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) # 精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる

    # 学習に用いるオプティマイザを返す関数を書く
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
    
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    #gpus=1,
    max_epochs=10,
    callbacks = [checkpoint]
)

model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

# ファインチューニングを行う
trainer.fit(model, dataloader_train, dataloader_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                          | Params | Mode
-----------------------------------------------------------------
0 | bert_sc | BertForSequenceClassification | 110 M  | eval
-----------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.497   Total estimated model params size (MB)
0         Modules in train mode
231       Modules in eval mode


Epoch 9: 100%|██████████| 24/24 [00:00<00:00, 33.52it/s, v_num=9]           

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 24/24 [00:01<00:00, 21.90it/s, v_num=9]


In [260]:
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)


ベストモデルのファイル:  /home/yozora98/Project/nlp2/model/epoch=9-step=240-v2.ckpt
ベストモデルの検証データに対する損失:  tensor(0.0229, device='cuda:0')


In [261]:
%load_ext tensorboard
%tensorboard --logdir="~/Project/nlp2/lightning_logs/"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 3252), started 3:53:21 ago. (Use '!kill 3252' to kill it.)

In [262]:
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

Restoring states from the checkpoint path at /home/yozora98/Project/nlp2/model/epoch=9-step=240-v2.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/yozora98/Project/nlp2/model/epoch=9-step=240-v2.ckpt


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 104.33it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy                   0.875
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy: 0.88


In [263]:
# Pytorch Lightning モデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(best_model_path)

# Transformers 対応のモデルを ./model_transformersに保存
model.bert_sc.save_pretrained('./model_transformers2')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [264]:
# モデルを読み込む
bert_sc = BertForSequenceClassification.from_pretrained('./model_transformers2')

In [265]:
# 推論 map or box
encoding = tokenizer(
    text_list,
    padding = 'longest',
    return_tensors='pt'
)
#encoding = { k: v.cuda() for k, v in encoding.items()}
#labels = torch.tensor(label_list).cuda()
encoding = { k: v for k, v in encoding.items()}
labels = torch.tensor(label_listB)
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits
labels_predictedA = scores.argmax(-1)
num_correct = (labels_predictedA==labels).sum().item()
accuracy = num_correct/labels.size(0)

print('size:')
print(scores.size())
print('predicted labels:')
print(labels_predictedA)
print('accuracy:')
print(accuracy)

size:
torch.Size([6, 9])
predicted labels:
tensor([0, 0, 0, 0, 0, 0])
accuracy:
0.6666666666666666


In [266]:
class Obj:
    def __init__(self, name):
        self.name = name

class MapObj(Obj):
    def __init__(self, name, obj_type='map', position=(0, 0), hp=100):
        super().__init__(name)
        self.type = obj_type
        self.position = position
        self.hp = hp

    def damage(self, damage):
        self.hp -= damage
        print(f'{damage}のダメージを与えた')
        if(self.hp <= 0):
            print('死亡!')
    
    def take(self):
        print(f'アイテム{self.name}をゲットした。')

    def find(self):
        print(f'マップ上{self.name}を発見した。')

            
class BoxObj(Obj):
    def __init__(self, name, obj_type='box', having=True):
        super().__init__(name)
        self.type = obj_type
        self.having = having
    
    def use(self):
        print(f'{self.name}を使用します。')
        self.having = False

    def find(self):
        print(f'Box内に{self.name}を発見した。')

m_obj_list = []
b_obj_list = []
m_obj1 = MapObj('グリフォン')
m_obj2 = MapObj('スライム', position=(5, 5))
m_obj_list.append(m_obj1)
m_obj_list.append(m_obj2)
b_obj1 = BoxObj('ポーション')
b_obj_list.append(b_obj1)

In [267]:
# マップやボックスに実際存在しているオブジェクト
for obj in m_obj_list:
    # map
    print(obj.name, obj.type)

print('------------')
for obj in b_obj_list:
    # box
    print(obj.name, obj.type)

グリフォン map
スライム map
------------
ポーション box


In [268]:
#class ItemBox:
#    def __init__(self):
#        self.item_list = []

In [269]:
def find_obj(is_box_obj=None, name=""):
    if is_box_obj is None:
        print('オブジェクトが見つかりませんでした。')
        return None
    elif is_box_obj == True:
        for obj in b_obj_list:
            if obj.name == name:
                print(f'ボックスオブジェクト[{name}]を見つけた。')
                return obj
        print(name, "が見つかりませんでした。(box)")
        return None
    else:
        for obj in m_obj_list:
            if obj.name == name:
                print(f'マップオブジェクト[{name}]を見つけた。')
                return obj
        print(name, "が見つかりませんでした。(map)")
        return None

In [270]:
class Character:
    def __init__(self, name, position=(9, 9)):
        self.name = name
        self.position = position
    
    def move_to(self, target_pos):
        print(f'{target_pos}に移動')
        self.position = target_pos

    def attack(self, target, damage):
        if type(target) is MapObj:
            target.damage(damage)
        else:
            print('攻撃可能なオブジェクトではありません。')

character = Character('hello')

音声入力による複雑な操作ができる。

In [273]:
# 行動生成？実行？の部分
# character position
print(f'プレイヤー{character.name}現在の位置', character.position)

# name_list == 仕様書
name_list = ['フェンリルAB','フェンリルA','ポーション', 'グリフォン', 'フェンリル', 'プリン', '大阪', 'スライム']
obj_name = ''
obj_indics = 2
for name in name_list:
    if name in text_list[obj_indics]:
        obj_name = name
        break
    else:
        continue

if obj_name == "":
    print('入力したオブジェクトの名前が仕様書にはありません。')

on_map = 0
in_box = 1
if labels_predictedA[obj_indics] == on_map: 
    # map obj
    target = find_obj(False, obj_name)
    if target:
        if labels_predictedB[obj_indics] == 6:
                print('unknown action!')
        elif labels_predictedB[obj_indics] == 0:
            # カテゴリー:移動
            character.move_to(target.position)  
        else:
            character.move_to(target.position)
            #アクション実行
            if labels_predictedB[obj_indics] == 1:
                character.attack(target, damage=10)
            elif labels_predictedB[obj_indics] == 2:
                target.take()
            elif labels_predictedB[obj_indics] == 4:
                target.find()
elif labels_predictedA[obj_indics] == in_box:
    # box_obj
    target = find_obj(True, obj_name)
    if target:
        if labels_predictedB[obj_indics] == 6:
                print('unknown action!')
        elif labels_predictedB[obj_indics] == 3:
            print('アイテムを所持していますか：', target.having)
            target.use()
            print('アイテムを所持していますか：', target.having)
        elif labels_predictedB[obj_indics] == 4:
            target.find()

プレイヤーhello現在の位置 (9, 9)
大阪 が見つかりませんでした。(map)


In [272]:
print(f'プレイヤー{character.name}現在の位置', character.position)

プレイヤーhello現在の位置 (9, 9)
